In [1]:
import pandas as pd
import sqlite3

# Load CSVs
providers = pd.read_csv('/content/providers_data.csv')
receivers = pd.read_csv('/content/receivers_data.csv')
food_listings = pd.read_csv('/content/food_listings_data.csv')
claims = pd.read_csv('/content/claims_data.csv')

# Preview
providers.head()


,Provider_ID,Name,Type,Address,City,Contact
0,1,Gonzales-Cochran,Supermarket,"74347 Christopher Extensions\nAndreamouth, OK ...",New Jessica,+1-600-220-0480
1,2,"Nielsen, Johnson and Fuller",Grocery Store,"91228 Hanson Stream\nWelchtown, OR 27136",East Sheena,+1-925-283-8901x6297
2,3,Miller-Black,Supermarket,"561 Martinez Point Suite 507\nGuzmanchester, W...",Lake Jesusview,001-517-295-2206
3,4,"Clark, Prince and Williams",Grocery Store,"467 Bell Trail Suite 409\nPort Jesus, IA 61188",Mendezmouth,556.944.8935x401
4,5,Coleman-Farley,Grocery Store,"078 Matthew Creek Apt. 319\nSaraborough, MA 53978",Valentineside,193.714.6577


In [2]:
# Create SQLite DB connection
conn = sqlite3.connect("food_waste.db")
cursor = conn.cursor()

# Load data into SQL tables
providers.to_sql("providers", conn, if_exists="replace", index=False)
receivers.to_sql("receivers", conn, if_exists="replace", index=False)
food_listings.to_sql("food_listings", conn, if_exists="replace", index=False)
claims.to_sql("claims", conn, if_exists="replace", index=False)

# Confirm tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())


[('providers',), ('receivers',), ('food_listings',), ('claims',)]


In [3]:
def run_query(query):
    return pd.read_sql(query, conn)

# 1. Food providers and receivers in each city
run_query("""
SELECT City, COUNT(*) as Num_Providers
FROM providers
GROUP BY City
""")

# 2. Type of provider contributing most
run_query("""
SELECT Type, COUNT(*) as Count
FROM providers
GROUP BY Type
ORDER BY Count DESC
LIMIT 1
""")

# 3. Contact info of providers in a specific city (example: Chennai)
run_query("""
SELECT Name, Contact
FROM providers
WHERE City = 'Chennai'
""")

# 4. Receivers with most food claims
run_query("""
SELECT r.Name, COUNT(*) as Num_Claims
FROM claims c
JOIN receivers r ON c.Receiver_ID = r.Receiver_ID
GROUP BY r.Name
ORDER BY Num_Claims DESC
LIMIT 5
""")

# 5. Total quantity of food available
run_query("""
SELECT SUM(Quantity) as Total_Quantity
FROM food_listings
""")

# 6. City with highest food listings
run_query("""
SELECT Location, COUNT(*) as Listings
FROM food_listings
GROUP BY Location
ORDER BY Listings DESC
LIMIT 1
""")

# 7. Most common food types
run_query("""
SELECT Food_Type, COUNT(*) as Count
FROM food_listings
GROUP BY Food_Type
ORDER BY Count DESC
""")

# 8. Claims per food item
run_query("""
SELECT Food_ID, COUNT(*) as Claims
FROM claims
GROUP BY Food_ID
ORDER BY Claims DESC
""")

# 9. Provider with highest successful claims
run_query("""
SELECT p.Name, COUNT(*) as Successful_Claims
FROM claims c
JOIN food_listings f ON c.Food_ID = f.Food_ID
JOIN providers p ON f.Provider_ID = p.Provider_ID
WHERE Status = 'Completed'
GROUP BY p.Name
ORDER BY Successful_Claims DESC
LIMIT 1
""")

# 10. Claim status distribution
run_query("""
SELECT Status, COUNT(*) as Count,
       ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM claims), 2) as Percentage
FROM claims
GROUP BY Status
""")

# 11. Avg quantity claimed per receiver
run_query("""
SELECT r.Name, AVG(f.Quantity) as Avg_Quantity
FROM claims c
JOIN food_listings f ON c.Food_ID = f.Food_ID
JOIN receivers r ON c.Receiver_ID = r.Receiver_ID
GROUP BY r.Name
""")

# 12. Most claimed meal type
run_query("""
SELECT Meal_Type, COUNT(*) as Claims
FROM claims c
JOIN food_listings f ON c.Food_ID = f.Food_ID
GROUP BY Meal_Type
ORDER BY Claims DESC
""")

# 13. Total quantity donated per provider
run_query("""
SELECT p.Name, SUM(f.Quantity) as Total_Donated
FROM food_listings f
JOIN providers p ON f.Provider_ID = p.Provider_ID
GROUP BY p.Name
ORDER BY Total_Donated DESC
""")


,Name,Total_Donated
0,Miller Inc,217
1,Barry Group,179
2,"Evans, Wright and Mitchell",158
3,Smith Group,150
4,Campbell LLC,145
...,...,...
623,Reyes and Sons,1
624,Norris-Townsend,1
625,Mcgee PLC,1
626,"Martinez, Armstrong and Carroll",1


In [4]:
# Add a new food listing
cursor.execute("""
INSERT INTO food_listings (Food_ID, Food_Name, Quantity, Expiry_Date, Provider_ID, Provider_Type, Location, Food_Type, Meal_Type)
VALUES (9999, 'Rice Packets', 10, '2025-06-10', 1, 'Restaurant', 'Chennai', 'Vegetarian', 'Lunch')
""")
conn.commit()

# Read it
run_query("SELECT * FROM food_listings WHERE Food_ID = 9999")

# Update quantity
cursor.execute("""
UPDATE food_listings SET Quantity = 20 WHERE Food_ID = 9999
""")
conn.commit()

# Delete it
cursor.execute("DELETE FROM food_listings WHERE Food_ID = 9999")
conn.commit()


In [5]:
!pip install streamlit pyngrok --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 110.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 100.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.5 MB/s eta 0:00:00


In [6]:
%%writefile app.py
import streamlit as st
import pandas as pd
import sqlite3

# DB connection
conn = sqlite3.connect("food_waste.db")

# Title
st.title("🥗 Local Food Wastage Management System")

# Tabs
tab1, tab2 = st.tabs(["🔍 Explore", "📊 SQL Analysis"])

with tab1:
    st.header("Food Listings Filter")

    city = st.selectbox("Select City", pd.read_sql("SELECT DISTINCT Location FROM food_listings", conn)['Location'])
    food_type = st.selectbox("Select Food Type", pd.read_sql("SELECT DISTINCT Food_Type FROM food_listings", conn)['Food_Type'])

    query = f"""
        SELECT Food_Name, Quantity, Expiry_Date, Meal_Type
        FROM food_listings
        WHERE Location = '{city}' AND Food_Type = '{food_type}'
    """
    results = pd.read_sql(query, conn)
    st.dataframe(results)

with tab2:
    st.header("Top 5 Cities with Most Food Listings")
    q = """
    SELECT Location, COUNT(*) as Listings
    FROM food_listings
    GROUP BY Location
    ORDER BY Listings DESC
    LIMIT 5
    """
    top_cities = pd.read_sql(q, conn)
    st.bar_chart(top_cities.set_index("Location"))


Writing app.py
